In [ ]:
import netgen.occ as occ
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from pathlib import Path
import os
import ngsolve as ngs

# create/obtain directories
CURRENT_DIR = Path(os.getcwd())
INFLOWS_DIR = CURRENT_DIR / "little_inflows"
STREAMLINES_DIR = CURRENT_DIR / "streamlines"
EDGES_DIR = CURRENT_DIR / "edges"
CONTOURS_DIR = CURRENT_DIR / "contours"
for fdir in [INFLOWS_DIR, STREAMLINES_DIR, EDGES_DIR, CONTOURS_DIR]:
    if not fdir.exists():
        fdir.mkdir()

In [ ]:
# generate occ points
def generate_mesh_from_boundaries(
    outer_size: tuple,
    inner_boundaries,
    maxh: float = None,
    margin: float = 0.8,
    # num_spline_points: int = 3,
):  
    # scale smallest outer boundary to 1
    scale = 1 / max(outer_size)

    # make outer boundary 10% larger to avoid meshing problems
    outer_size_sc = tuple([(1 + margin) * x * scale for x in outer_size])
    print(f"Outer size: {outer_size_sc}")
    
    face = occ.Rectangle(*outer_size_sc).Face()

    # offset to ensure innner boundaries are centered in the outer boundary + flip y
    offset_x = margin / 2 * outer_size[0] * scale
    offset_y = margin / 2 * outer_size[1] * scale
    inner_boundaries = [
        [(x*scale + offset_x, outer_size_sc[1] - (y*scale + offset_y)) for x, y in inner_boundary]
        for inner_boundary in inner_boundaries
    ]

    for inner_bnd in inner_boundaries:
        inner_segs = [
            occ.SplineApproximation(
                [occ.Pnt(*inner_bnd[i], 0), occ.Pnt(*inner_bnd[i + 1], 0)],
                deg_min=2,
                deg_max=3,
            )
            for i in range(len(inner_bnd) - 1)
        ] + [
            occ.SplineApproximation(
                [occ.Pnt(*inner_bnd[-1], 0), occ.Pnt(*inner_bnd[0], 0)],
                deg_min=2,
                deg_max=3,
            )
        ]
        inner_wire = occ.Wire(inner_segs)
        inner_face = occ.Face(inner_wire)
        face -= inner_face

    # name edges
    face.edges.name = "inner_wall"
    face.edges.Max(occ.Y).name = "lid"
    face.edges.Min(occ.Y).name = "floor"
    face.edges.Max(occ.X).name = "right"
    face.edges.Min(occ.X).name = "left"

    # split inner wall into left, right, top and bottom
    sorted_edges_x = face.edges.Sorted(occ.X)
    inner_x_min = offset_x
    inner_x_max = offset_x + outer_size[0]*scale
    sorted_edges_y = face.edges.Sorted(occ.Y)
    inner_y_min = offset_y
    inner_y_max = offset_y + outer_size[1]*scale

    inner_edge_margin = 0.1
    inner_dist_x = inner_edge_margin * outer_size[0] * scale
    inner_dist_y = inner_edge_margin * outer_size[1] * scale
    for edge in sorted_edges_x:
        xval = edge.center[0]
        if edge.name == "inner_wall":
            if inner_x_min < xval < inner_x_min + inner_dist_x:
                edge.name = "inner_wall_left"
            if inner_x_max - inner_dist_x < xval < inner_x_max:
                edge.name = "inner_wall_right"
    for edge in sorted_edges_y:
        yval = edge.center[1]
        if edge.name == "inner_wall":
            if inner_y_min < yval < inner_y_min + inner_dist_y:
                edge.name = "inner_wall_bottom"
            if inner_y_max - inner_dist_y < yval < inner_y_max:
                edge.name = "inner_wall_top"

    geom = occ.OCCGeometry(face, dim=2)
    if maxh is None:
        maxh = 0.03 * max(outer_size_sc)

    print(f"Generating mesh with maxh={maxh}")
    return ngs.Mesh(geom.GenerateMesh(maxh=maxh))


outer_boundary = [(0, 0), (2, 0), (2, 2), (0, 2)]

# Define hole(s) (clockwise)
holes = [
    [(0.5, 0.5), (1, 0.5), (1, 1), (0.5, 1)],  # First hole
    [(1.2, 1.2), (1.6, 1.2), (1.6, 1.6), (1.2, 1.6)],  # Second hole
    [(0.8, 0.8), (1.2, 0.8), (1.2, 1.2), (0.8, 1.2)],  # Third hole
]

# Generate the mesh using OCC functions
mesh_simple = generate_mesh_from_boundaries((2, 2), holes, margin=0.9)
Draw(mesh_simple, "mesh")

In [ ]:
contours = []
num_contours = 1
key = "cut_out_turbine"
for file in (CONTOURS_DIR / key).iterdir():
    contours.append(np.loadtxt(file))
edges = plt.imread(EDGES_DIR / f"{key}.png")
image_height, image_width = np.loadtxt(EDGES_DIR / f"{key}.txt")
domain_size = (image_width, image_height)
print(domain_size)
margin = 0.5
mesh_contour = generate_mesh_from_boundaries(domain_size, contours, margin=margin)
Draw(mesh_contour)

In [ ]:
nu = 1e-3 # viscosity
tend = 0.3  #seconds
dt = 0.0001
mesh = mesh_contour
domain_width = ngs.Integrate(1, mesh.Boundaries("lid"))
domain_length = ngs.Integrate(1, mesh.Boundaries("left"))

inner_wall_right_length = ngs.Integrate(1, mesh.Boundaries("inner_wall_right"))
print(f"Length of the lid & floor : {domain_width}")
print(f"Length of left & right wall: {domain_length}")
print(f"Length of the inner wall right: {inner_wall_right_length}")
print(f"Length of the inner wall left: {ngs.Integrate(1, mesh.Boundaries('inner_wall_left'))}")
print(f"Length of the inner wall top: {ngs.Integrate(1, mesh.Boundaries('inner_wall_top'))}")
print(f"Length of the inner wall bottom: {ngs.Integrate(1, mesh.Boundaries('inner_wall_bottom'))}")

In [ ]:
V = ngs.VectorH1(
    mesh,
    order=3,
    dirichlet="right|floor|lid|inner_wall|inner_wall_left|inner_wall_right|inner_wall_top|inner_wall_bottom",
)


Q = ngs.H1(mesh, order=2)


N = ngs.NumberSpace(mesh)


X = V * Q * N


u, p, lam = X.TrialFunction()


v, q, mu = X.TestFunction()

In [ ]:
# gridfunction for the solution
gfu = ngs.GridFunction(X)

# parabolic lid flow
scaling = 0.0001
u_left = ngs.CoefficientFunction(
    (8 * ngs.y * (domain_length - ngs.y) / domain_length**2, 0)
)
u_inner_wall_top = ngs.CoefficientFunction((scaling, 0))
u_inner_wall_bottom = ngs.CoefficientFunction((scaling, 0))

u_inner_wall_right = ngs.CoefficientFunction(
    (
        scaling
        * 8
        * ngs.y
        * (inner_wall_right_length - ngs.y)
        / inner_wall_right_length**2,
        0,
    )
)
u_top = ngs.CoefficientFunction(
    (0, -8 * ngs.x * (domain_width - ngs.x) / domain_width**2)
)
u_right = ngs.CoefficientFunction(
    (-8 * ngs.y * (domain_length - ngs.y) / domain_length**2, 0)
)
u_floor = ngs.CoefficientFunction(
    (0, 8 * ngs.x * (domain_width - ngs.x) / domain_width**2)
)

# add similar conditions on all boundaries
# gfu.components[0].Set(u_left, definedon=mesh.Boundaries("left"))
# gfu.components[0].Set(u_inner_wall_top, definedon=mesh.Boundaries("inner_wall_top"))
# gfu.components[0].Set(u_inner_wall_bottom, definedon=mesh.Boundaries("inner_wall_bottom"))
# gfu.components[0].Set(u_inner_wall_right, definedon=mesh.Boundaries("inner_wall_right"))
# gfu.components[0].Set(u_top, definedon=mesh.Boundaries("lid"))
gfu.components[0].Set(u_right, definedon=mesh.Boundaries("right"))
# gfu.components[0].Set(u_floor, definedon=mesh.Boundaries("floor"))

ngs.Redraw()
ngs.Draw(gfu.components[0], mesh, "u")

In [ ]:
stokes = (
    nu * ngs.InnerProduct(ngs.grad(u), ngs.grad(v))
    + ngs.InnerProduct(ngs.grad(u) * u, v)
    - ngs.div(u) * q
    - ngs.div(v) * p
    - lam * q
    - mu * p
) * ngs.dx



a = ngs.BilinearForm(X, symmetric=True)


a += stokes


a.Assemble()


f = ngs.LinearForm(X)


f.Assemble()


res = f.vec - a.mat * gfu.vec


inv_stokes = a.mat.Inverse(X.FreeDofs())


gfu.vec.data += inv_stokes * res

In [ ]:
sceneu = Draw(gfu.components[0], mesh, "u", vectors={"grid_size": 30})
scenep = Draw(gfu.components[1], mesh, "p")

In [ ]:
# matrix for implicit part of IMEX(1) scheme:
mstar = ngs.BilinearForm(X)
mstar += ngs.InnerProduct(u, v) * ngs.dx + dt * stokes
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs())

conv = ngs.BilinearForm(X, nonassemble=True)
conv += (ngs.Grad(u) * u) * v * ngs.dx


gfut = ngs.GridFunction(gfu.space, multidim=0)

# implicit Euler/explicit Euler splitting method:
gfut.AddMultiDimComponent(gfu.vec)
t = 0
cnt = 0
while t < tend - 0.5 * dt:
    print("\rt=", t, end="")

    conv.Assemble()
    res = (a.mat + conv.mat) * gfu.vec
    gfu.vec.data -= dt * inv * res

    t = t + dt
    cnt += 1
    sceneu.Redraw()
    scenep.Redraw()
    if cnt % 50 == 0:
        gfut.AddMultiDimComponent(gfu.vec)

In [ ]:
Draw(
    gfut.components[0],
    mesh,
    interpolate_multidim=True,
    animate=True,
    min=0,
    max=1.9,
    autoscale=False,
    vectors={"grid_size": 30},
)
Draw(
    gfut.components[1],
    mesh,
    interpolate_multidim=True,
    animate=True,
    min=-0.5,
    max=1,
    autoscale=False,
)

In [ ]:
u, v = gfu.components[0].components
n = 100  # Grid resolution

xvals = np.linspace(0, domain_width, n)
yvals = np.linspace(0, domain_length, n)
Xg, Yg = np.meshgrid(xvals, yvals)


U = np.zeros((n, n))
V = np.zeros((n, n))
M = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        # check if in domain
        try:
            pnt = mesh(xvals[i], yvals[j])
            U[j, i] = u(pnt)
            V[j, i] = v(pnt)
            M[j, i] = np.sqrt(U[j, i] ** 2 + V[j, i] ** 2)
        except:
            continue

In [ ]:
# Plot streamlines
dpi = 1000
figheight = image_height * (1 + margin) / dpi
figwidth = image_width  * (1 + margin) / dpi
fig, ax = plt.subplots(1, 1, figsize=(figwidth, figheight), dpi=dpi)
ax.streamplot(
    Xg,
    Yg,
    U,
    V,
    density=5,
    linewidth=0.5,
    arrowsize=0.5,
    arrowstyle="-|>",
    zorder=1,
    cmap="plasma",
    color=M,
)

# plot magnitude of velocity
ax.set_axis_off()
plt.show()

# save just the plot (without axes and labels)
fp = STREAMLINES_DIR / f"{key}.png"
fig.savefig(fp, bbox_inches="tight", pad_inches=0, transparent=True)

In [ ]:
# merge image edges and streamlines into one image
plt.style.use("dark_background")
fig, ax = plt.subplots(1, 1, figsize=(figwidth, figheight), dpi=dpi)

streamlines = plt.imread(STREAMLINES_DIR / f"{key}.png")
print(f"edge dimensions: {edges.shape}, ratio: {edges.shape[0] / edges.shape[1]}")
print(f"streamline dimensions: {streamlines.shape}, ratio: {streamlines.shape[0] / streamlines.shape[1]}")

x_offset = image_width * margin / 2
y_offset = image_height * margin / 2
xmin, xmax, ymin, ymax = (
    x_offset,
    domain_width - x_offset,
    y_offset,
    domain_length - y_offset,
)
factor = 1
plt.imshow(
    edges,
    cmap="gray",
    zorder=0,
    extent=[xmin, xmin + image_width, ymin, ymin + image_height],
    origin="upper",
)
plt.imshow(streamlines, zorder=1, extent=[0, (1+margin)*image_width, 0, (1 + margin)*image_height], origin="upper")

plt.axis("off")

plt.savefig(INFLOWS_DIR / f"{key}.png", bbox_inches="tight", pad_inches=0,dpi=2000)